# Team Ratings

In [1]:
# Local libraries
import Tools.ratings_utils as ru

YEAR = 2023
FILENAME = f"Data/data_{YEAR}.json"
TOURNAMENT_FILENAME = f"Data/tournament_{YEAR}.csv"


score_df = ru.set_rating_data_frame(filename=FILENAME)

## Massey Ratings

In [2]:
massey_ratings = ru.calculate_massey_ratings(score_df=score_df,
                                             debug=False)

_ = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                           ratings=massey_ratings)

Round: 1 / Round of 64 - Correct picks: 24 out of 32
Round: 2 / Round of 32 - Correct picks: 9 out of 16
Round: 3 / Sweet 16 - Correct picks: 2 out of 8
Round: 4 / Elite 8 - Correct picks: 0 out of 4
Round: 5 / Final 4 - Correct picks: 0 out of 2
Round: 6 / Championship - Correct picks: 0 out of 1

Total correct picks in tournament: 35 out of 63


# Colley Ratings

In [3]:
colley_ratings = ru.calculate_colley_ratings(score_df=score_df,
                                             debug=False)

_ = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                           ratings=colley_ratings)

Round: 1 / Round of 64 - Correct picks: 22 out of 32
Round: 2 / Round of 32 - Correct picks: 9 out of 16
Round: 3 / Sweet 16 - Correct picks: 2 out of 8
Round: 4 / Elite 8 - Correct picks: 0 out of 4
Round: 5 / Final 4 - Correct picks: 0 out of 2
Round: 6 / Championship - Correct picks: 0 out of 1

Total correct picks in tournament: 33 out of 63


# Elo Ratings

In [4]:
adj_elo_ratings = ru.calculate_elo_ratings(score_df=score_df,
                                           K=30,
                                           debug=False,
                                           adjust_K=True)

_ = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                           ratings=adj_elo_ratings)

Round: 1 / Round of 64 - Correct picks: 17 out of 32
Round: 2 / Round of 32 - Correct picks: 7 out of 16
Round: 3 / Sweet 16 - Correct picks: 2 out of 8
Round: 4 / Elite 8 - Correct picks: 0 out of 4
Round: 5 / Final 4 - Correct picks: 0 out of 2
Round: 6 / Championship - Correct picks: 0 out of 1

Total correct picks in tournament: 26 out of 63


In [5]:
elo_ratings = ru.calculate_elo_ratings(score_df=score_df,
                                       K=30,
                                       debug=False,
                                       adjust_K=False)

_ = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                           ratings=elo_ratings)

Round: 1 / Round of 64 - Correct picks: 14 out of 32
Round: 2 / Round of 32 - Correct picks: 6 out of 16
Round: 3 / Sweet 16 - Correct picks: 3 out of 8
Round: 4 / Elite 8 - Correct picks: 1 out of 4
Round: 5 / Final 4 - Correct picks: 0 out of 2
Round: 6 / Championship - Correct picks: 0 out of 1

Total correct picks in tournament: 24 out of 63


# SRS Ratings

In [6]:
srs_ratings = ru.compile_srs_ratings(filename=FILENAME,
                                     debug=False)

_ = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                           ratings=srs_ratings)

KeyError: 'Fairleigh Dickinson-Florham'

# Combined Ratings

In [ ]:
import numpy as np

def normalize_ratings(ratings: dict, weight: float=1):
    """Normalize rating values between 0 and 1

    Args:
        ratings (dict): dictionary of ratings
        weight (float): scaled value applied to normalized rating systems
        
    Returns:
        normalized_dict (dict): dictionary of normalized ratings
    """

    values = list(ratings.values())  # [15, 20, 30]
    
    # Min-Max Scaling
    min_val = min(ratings.values())
    max_val = max(ratings.values())
    
    normalized_dict = {k: weight * (v - min_val) / (max_val - min_val) for k, v in ratings.items()}
    normalized_values = list(normalized_dict.values())  # [15, 20, 30]
    
    return normalized_dict


In [ ]:
step = 0.2
iterations = int((1 / step) + 1)
total_iterations = 0

weight_dict = {'Massey': [], 'Colley': [], 'Adjusted Elo': [], 'Elo': [], 'Correct': []}

for i in range(iterations):
    for j in range(iterations):
        for k in range(iterations):
            for l in range(iterations):
                total_iterations += 1
                
                w1 = i * step
                w2 = j * step
                w3 = k * step
                w4 = l * step
    
                # Normalize weights
                # min_w = min([w1, w2, w3])
                # max_w = max([w1, w2, w3])
                # if max_w - min_w > 0:
                #     w1 = (w1 - min_w) / (max_w - min_w)
                #     w2 = (w2 - min_w) / (max_w - min_w)
                #     w3 = (w3 - min_w) / (max_w - min_w)
    
                normalized_massey_ratings = normalize_ratings(ratings=massey_ratings, weight=w1)
                normalized_colley_ratings = normalize_ratings(ratings=colley_ratings, weight=w2)
                normalized_adj_elo_ratings = normalize_ratings(ratings=adj_elo_ratings, weight=w3)
                normalized_elo_ratings = normalize_ratings(ratings=elo_ratings, weight=w4)
                
                combined_ratings = {}
                
                for key, v in normalized_massey_ratings.items():
                    combined_ratings[key] = v
                for key, v in normalized_colley_ratings.items():
                    combined_ratings[key] += v
                for key, v in normalized_adj_elo_ratings.items():
                    combined_ratings[key] += v
                for key, v in normalized_elo_ratings.items():
                    combined_ratings[key] += v
                
                total_correct_picks = ru.simulate_tournament(filename=TOURNAMENT_FILENAME,
                                                             ratings=combined_ratings,
                                                             debug=False)
            
                weight_dict['Massey'].append(w1)
                weight_dict['Colley'].append(w2)
                weight_dict['Adjusted Elo'].append(w3)
                weight_dict['Elo'].append(w4)
                weight_dict['Correct'].append(total_correct_picks)

max_correct = max(weight_dict['Correct'])
print(f"Max correct picks: {max_correct} in {total_iterations} iterations")

num_max = 0

for i in range(len(weight_dict['Correct'])):

    if weight_dict['Correct'][i] == max_correct:
        num_max += 1
        print(f"{num_max}. Massey: {weight_dict['Massey'][i]:.3f}, Colley: {weight_dict['Colley'][i]:.3f}, Adjusted Elo: {weight_dict['Adjusted Elo'][i]:.3f}, Elo: {weight_dict['Elo'][i]:.3f}")


# Test code

In [ ]:
# Currently unused (save for Massey, Colley, Elo Ratings)
tournament = Tournament.Tournament(url=TOURNAMENT_URL,
                                   debug=True)

In [ ]:
# # Check results manually
# import csv

# # Convert dictionary to a CSV-friendly format
# with open("teams.csv", mode="w", newline="") as file:
#     writer = csv.writer(file)

#     keys = list(tourney_dict.keys())
#     writer.writerow(keys)  # Header

#     for i in range(len(tourney_dict[keys[0]])):
#         row = []
#         for key in keys:
#             row.append(tourney_dict[key][i])
#         writer.writerow(row)  # Combine team name with stats